# ML2025 Retrieval Augmented Generation with Agents

## Environment Setup

In this section, we install the necessary python packages and download model weights of the quantized version of LLaMA 3.1 8B. Also, download the dataset. Note that the model weight is around 8GB.

In [1]:
!python3 -m pip install --no-cache-dir llama-cpp-python==0.3.4 --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu122
!python3 -m pip install googlesearch-python bs4 charset-normalizer requests-html lxml_html_clean

from pathlib import Path
if not Path('./Meta-Llama-3.1-8B-Instruct-Q8_0.gguf').exists():
    !wget https://huggingface.co/bartowski/Meta-Llama-3.1-8B-Instruct-GGUF/resolve/main/Meta-Llama-3.1-8B-Instruct-Q8_0.gguf
if not Path('./public.txt').exists():
    !wget https://www.csie.ntu.edu.tw/~ulin/public.txt
if not Path('./private.txt').exists():
    !wget https://www.csie.ntu.edu.tw/~ulin/private.txt

Looking in indexes: https://pypi.org/simple, https://abetlen.github.io/llama-cpp-python/whl/cu122
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 445.2/445.2 MB 155.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 44.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 14.1
    Uninstalling websockets-14.1:
      Successfully uninstalled websockets-14.1
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.3.0
    Uninstalling urllib3-2.3.0:
      Successfully uninstalled urllib3-2.3.0
ERROR: pip's depende

In [2]:
import torch
if not torch.cuda.is_available():
    raise Exception('You are not using the GPU runtime. Change it first or you will suffer from the super slow inference speed!')
else:
    print('You are good to go!')

You are good to go!


## Prepare the LLM and LLM utility function

By default, we will use the quantized version of LLaMA 3.1 8B. You can also try out different LLM models.

In the following code block, we will load the downloaded LLM model weights onto the GPU first.
Then, we implemented the generate_response() function so that you can get the generated response from the LLM model more easily.

You can ignore "llama_new_context_with_model: n_ctx_per_seq (16384) < n_ctx_train (131072) -- the full capacity of the model will not be utilized" warning.

In [3]:
from llama_cpp import Llama

# Load the model onto GPU
llama3 = Llama(
    "./Meta-Llama-3.1-8B-Instruct-Q8_0.gguf",
    verbose=False,
    n_gpu_layers=-1,
    n_ctx=16384,    # This argument is how many tokens the model can take. The longer the better, but it will consume more memory. 16384 is a proper value for a GPU with 16GB VRAM.
)

def generate_response(_model: Llama, _messages: str) -> str:
    '''
    This function will inference the model with given messages.
    '''
    _output = _model.create_chat_completion(
        _messages,
        stop=["<|eot_id|>", "<|end_of_text|>"],
        max_tokens=512,    # This argument is how many tokens the model can generate.
        temperature=0,      # This argument is the randomness of the model. 0 means no randomness. You will get the same result with the same input every time. You can try to set it to different values.
        repeat_penalty=2.0,
    )["choices"][0]["message"]["content"]
    return _output

llama_new_context_with_model: n_ctx_per_seq (16384) < n_ctx_train (131072) -- the full capacity of the model will not be utilized


## Search Tool

You can use this tool to search for the relevant **web pages** for the given question. The search tool can be integrated in the following sections.

In [4]:
from typing import List
from googlesearch import search as _search
from bs4 import BeautifulSoup
from charset_normalizer import detect
import asyncio
from requests_html import AsyncHTMLSession
import urllib3
urllib3.disable_warnings()

async def worker(s:AsyncHTMLSession, url:str):
    try:
        header_response = await asyncio.wait_for(s.head(url, verify=False), timeout=10)
        if 'text/html' not in header_response.headers.get('Content-Type', ''):
            return None
        r = await asyncio.wait_for(s.get(url, verify=False), timeout=10)
        return r.text
    except:
        return None

async def get_htmls(urls):
    session = AsyncHTMLSession()
    tasks = (worker(session, url) for url in urls)
    return await asyncio.gather(*tasks)

async def search(keyword: str, n_results: int=3) -> List[str]:
    '''
    This function will search the keyword and return the text content in the first n_results web pages.
    Warning: You may suffer from HTTP 429 errors if you search too many times in a period of time. This is unavoidable and you should take your own risk if you want to try search more results at once.
    The rate limit is not explicitly announced by Google, hence there's not much we can do except for changing the IP or wait until Google unban you (we don't know how long the penalty will last either).
    '''
    keyword = keyword[:100]
    # First, search the keyword and get the results. Also, get 2 times more results in case some of them are invalid.
    results = list(_search(keyword, n_results * 2, lang="zh", unique=True))
    # Then, get the HTML from the results. Also, the helper function will filter out the non-HTML urls.
    results = await get_htmls(results)
    # Filter out the None values.
    results = [x for x in results if x is not None]
    # Parse the HTML.
    results = [BeautifulSoup(x, 'html.parser') for x in results]
    # Get the text from the HTML and remove the spaces. Also, filter out the non-utf-8 encoding.
    results = [''.join(x.get_text().split()) for x in results if detect(x.encode()).get('encoding') == 'utf-8']
    # Return the first n results.
    return results[:n_results]

In [5]:
from typing import List
from googlesearch import search as _search
from bs4 import BeautifulSoup
from charset_normalizer import detect
import asyncio
from requests_html import AsyncHTMLSession
import urllib3
urllib3.disable_warnings()

async def worker(s:AsyncHTMLSession, url:str):
    try:
        header_response = await asyncio.wait_for(s.head(url, verify=False), timeout=10)
        if 'text/html' not in header_response.headers.get('Content-Type', ''):
            return None
        r = await asyncio.wait_for(s.get(url, verify=False), timeout=10)
        return r.text
    except:
        return None

async def get_htmls(urls):
    session = AsyncHTMLSession()
    tasks = (worker(session, url) for url in urls)
    return await asyncio.gather(*tasks)

async def search(keywords: str, n_results: int=3) -> List[str]:
    '''
    This function will search the keyword and return the text content in the first n_results web pages.

    Warning: You may suffer from HTTP 429 errors if you search too many times in a period of time. This is unavoidable and you should take your own risk if you want to try search more results at once.
    The rate limit is not explicitly announced by Google, hence there's not much we can do except for changing the IP or wait until Google unban you (we don't know how long the penalty will last either).
    '''
    keywords = keywords[:100]
    filtered = []

    # # Split keywords by comma (you can adjust if your agent uses different separator)
    # kw_list = [kw.strip() for kw in keywords.split(",") if kw.strip()]
    # # Try Wikipedia for each keyword
    # for main_kw in kw_list:
    #     wiki_url = f"https://zh.wikipedia.org/wiki/{main_kw}"
    #     wiki_htmls = await get_htmls([wiki_url])
    #     if wiki_htmls and wiki_htmls[0]:
    #         soup = BeautifulSoup(wiki_htmls[0], 'html.parser')
    #         text = ''.join(soup.get_text().split())
    #         filtered.append((wiki_url, text[:600]))  # Only first 600 chars for context
    #         if len(filtered) >= n_results:
    #             return filtered
    
    
    # 
    # Step 2: If Wikipedia is not enough, use Google search as before
    # First, search the keyword and get the results. Also, get 2 times more results in case some of them are invalid.
    urls = list(_search(keywords, n_results * 2, lang="zh", unique=True))
    print("Google 搜尋到的 URL】", urls)
    # Then, get the HTML from the results. Also, the helper function will filter out the non-HTML urls.
    # results = await get_htmls(results)
    html_results = await get_htmls(urls)
    for url, html in zip(urls, html_results):
        print(f"【檢索網址】: {url} {'[成功]' if html else '[失敗]'}")
        if html is None:
            continue
        try:
            soup = BeautifulSoup(html, 'html.parser')
            text = ''.join(soup.get_text().split())
            filtered.append((url, text[:900]))
            # stop when wnough
            if len(filtered) >= n_results:
                break
        except Exception:
            continue
    return filtered

## Test the LLM inference pipeline

In [6]:
# You can try out different questions here.
test_question='請問誰是 Taylor Swift？'

messages = [
    {"role": "system", "content": "你是 LLaMA-3.1-8B，是用來回答問題的 AI。使用中文時只會使用繁體中文來回問題。"},    # System prompt
    {"role": "user", "content": test_question}, # User prompt
]

print(generate_response(llama3, messages))

泰勒絲（Taylor Swift）是一位美國歌手、詞曲作家和音樂製作人。她出生於1989年，來自田納西州。她的音乐风格从乡村摇滚发展到流行搖擺，並且她被誉为当代最成功的女艺人的之一。

泰勒絲早期在鄉郊小鎮演唱會時開始發展音樂事業，她推出了多張專輯，包括《Taylor Swift》、《Fearless》，以及後來更為知名的大熱作如 《1989》（2014年）、_reputation（）和 _Lover （）。她的歌曲經常探討愛情、友誼及自我成長等主題。

泰勒絲獲得了許多獎項，包括13座格萊美奖，並且是史上最快達到百萬銷量的女藝人之一。


## Agents

You can use this class to create agents that can interact with the LLM model. The Agent class has the following attributes and methods:
- Attributes:
    - role_description: The role of the agent. For example, if you want this agent to be a history expert, you can set the role_description to "You are a history expert. You will only answer questions based on what really happened in the past. Do not generate any answer if you don't have reliable sources.".
    - task_description: The task of the agent. For example, if you want this agent to answer questions only in yes/no, you can set the task_description to "Please answer the following question in yes/no. Explanations are not needed."
    - llm: Just an indicator of the LLM model used by the agent.
- Method:
    - inference: This method takes a message as input and returns the generated response from the LLM model. The message will first be formatted into proper input for the LLM model. (This is where you can set some global instructions like "Please speak in a polite manner" or "Please provide a detailed explanation".) The generated response will be returned as the output.

In [7]:
class LLMAgent():
    def __init__(self, role_description: str, task_description: str, llm:str="bartowski/Meta-Llama-3.1-8B-Instruct-GGUF"):
        self.role_description = role_description   # Role means who this agent should act like. e.g. the history expert, the manager......
        self.task_description = task_description    # Task description instructs what task should this agent solve.
        self.llm = llm  # LLM indicates which LLM backend this agent is using.
    def inference(self, message:str) -> str:
        if self.llm == 'bartowski/Meta-Llama-3.1-8B-Instruct-GGUF': # If using the default one.
            # TODO: Design the system prompt and user prompt here.
            # Format the messsages first.
            messages = [
                {"role": "system", "content": f"{self.role_description}\n請使用繁體中文回答，並且語氣要有禮貌"},  # Hint: you may want the agents to speak Traditional Chinese only.
                {"role": "user", "content": f"{self.task_description}\n【使用者問題】{message}"}, # Hint: you may want the agents to clearly distinguish the task descriptions and the user messages. A proper seperation text rather than a simple line break is recommended.
            ]
            return generate_response(llama3, messages)
        else:
            # TODO: If you want to use LLMs other than the given one, please implement the inference part on your own.
            return ""

In [8]:
# TODO: Design the role and task description for each agent.

# This agent may help you filter out the irrelevant parts in question descriptions.
question_extraction_agent = LLMAgent(
    role_description="你是一位問題拆解專家，能將複雜或冗長的問題轉換成一個簡單、直接、聚焦的問句，只保留與核心問題直接相關的資訊，不要加入多餘解釋。",
    task_description="請將下列問題轉換成單一句簡明問句，只保留核心重點：",
)

# This agent may help you extract the keywords in a question so that the search tool can find more accurate results.
keyword_extraction_agent = LLMAgent(
    # role_description="你是一位關鍵字提取專家，能從用戶的問題中擷取最能代表問題主題的中英文關鍵字，以便進行有效的資料檢索",
    # task_description="請從下列問題中提取三到五個最重要的關鍵字（以逗號分隔）：",
    role_description="你是一位關鍵字提取專家，能從問題中擷取所有有助於搜尋答案的完整關鍵詞或詞組，如果兩個或多個詞組合起來更有助於搜尋，請一起列出，不要拆分不要加解釋，只輸出關鍵詞。",
    task_description="請從下列問題中，提取三到五個最重要的完整關鍵詞或詞組：",
)

# This agent is the core component that answers the question.
qa_agent = LLMAgent(
    role_description="你是資深問題回答專家，請只根據下方檢索內容，使用繁體中文給出準確且簡明的答案。若檢索內容無明確答案，請誠實回答「根據檢索內容無法判斷」。不要靠自己的知識猜測，也不要多餘前後說明。如需條列重點，可用條列式簡潔列出。",
    task_description="根據提供的檢索內容，請直接回答下列問題：",
)

## RAG pipeline

Also, there might be more heuristics (e.g. classifying the questions based on their lengths, determining if the question need a search or not, reconfirm the answer before returning it to the user......) that are not shown in the flow charts. You can use your creativity to come up with a better solution!

- Naive approach (simple baseline)

    ![](https://www.csie.ntu.edu.tw/~ulin/naive.png)

- Naive RAG approach (medium baseline)

    ![](https://www.csie.ntu.edu.tw/~ulin/naive_rag.png)

- RAG with agents (strong baseline)

    ![](https://www.csie.ntu.edu.tw/~ulin/rag_agent.png)

In [9]:
async def pipeline(question: str) -> str:
    # TODO: Implement your pipeline.
    # Currently, it only feeds the question directly to the LLM.
    # You may want to get the final results through multiple inferences.
    # Just a quick reminder, make sure your input length is within the limit of the model context window (16384 tokens), you may want to truncate some excessive texts.
    return qa_agent.inference(question)

In [10]:
async def pipeline(question: str) -> str:
    # TODO: Implement your pipeline.
    # Currently, it only feeds the question directly to the LLM.
    # You may want to get the final results through multiple inferences.
    # Just a quick reminder, make sure your input length is within the limit of the model context window (16384 tokens), you may want to truncate some excessive texts.
    print("原始問題:", question)
    simplified_question = question_extraction_agent.inference(question)
    print("精簡問題:", simplified_question)
    keywords = keyword_extraction_agent.inference(simplified_question)
    print("關鍵字:", keywords)
    search_results = await search(keywords, n_results = 2)
    print("搜尋結果:", search_results)
    print("【實際搜尋的 URL】", [url for url, _ in search_results])
    # context = "\n".join(search_results[:1])
    context = "\n".join([f"來源：{url}\n{text}" for url, text in search_results])
    qa_input = f"【問題】{simplified_question}\n【檢索內容】{context}"
    print("QA 輸入內容:", qa_input)
    final_answer = qa_agent.inference(qa_input)
    print("最終答案:", final_answer)
    return final_answer
    # return qa_agent.inference(question)

## Answer the questions using your pipeline!

Since Colab has usage limit, you might encounter the disconnections. The following code will save your answer for each question. If you have mounted your Google Drive as instructed, you can just rerun the whole notebook to continue your process.

In [11]:
test_question = "2005 播出的電視劇《終極⼀班》中，有⼀個⾼中⽣戰⼒排⾏榜，稱為「KO榜」，該榜榜⾸為？"
result = await pipeline(test_question)
print("\n[Pipeline 回答結果]:\n", result)

原始問題: 2005 播出的電視劇《終極⼀班》中，有⼀個⾼中⽣戰⼒排⾏榜，稱為「KO榜」，該榜榜⾸為？
精簡問題: 您想問的是《終極⼀班》中的「KO榜」第一名是誰？
關鍵字: 終極⼀班 KO榜
Google 搜尋到的 URL】 ['https://zh.wikipedia.org/zh-tw/KO%E6%A6%9C', 'https://baike.baidu.com/item/k.o%E6%A6%9C/6797339', 'https://zh.wikipedia.org/wiki/%E7%B5%82%E6%A5%B5%E4%B8%80%E7%8F%AD', 'https://www.threads.com/@coldboxbei/post/DBfMNzaTsp8']
【檢索網址】: https://zh.wikipedia.org/zh-tw/KO%E6%A6%9C [成功]
【檢索網址】: https://baike.baidu.com/item/k.o%E6%A6%9C/6797339 [失敗]
【檢索網址】: https://zh.wikipedia.org/wiki/%E7%B5%82%E6%A5%B5%E4%B8%80%E7%8F%AD [成功]
搜尋結果: [('https://zh.wikipedia.org/zh-tw/KO%E6%A6%9C', '終極宇宙-維基百科，自由的百科全書跳至內容主選單主選單移至側邊欄隱藏導覽首頁分類索引特色內容新聞動態近期變更隨機條目說明說明維基社群方針與指引互助客棧知識問答字詞轉換IRC即時聊天聯絡我們關於維基百科特殊頁面搜尋搜尋外觀資助維基百科建立帳號登入個人工具資助維基百科建立帳號登入目次移至側邊欄隱藏序言1起源2世界觀切換世界觀子章節2.1第一時期2.2第二時期3劇情發展與風格切換劇情發展與風格子章節3.1主線劇情3.2分支劇情3.3終極宇宙時間軸預估3.4原作與翻拍4名詞解釋切換名詞解釋子章節4.1人物種類相關說明4.1.1異能行者4.1.2其他人物名稱4.2指數4.3KO榜4.3.1《終極一班》4.3.1.1榜內4.3.1.2榜外4.3.2《終極一班3》4.3.2.1舊·榜內4.3.2.2新·榜內4.3.2.3榜外4.3.3《終極一班4》4.3.3.1榜內＆未參榜4.3.3.2榜外4.4其他名詞5主要演員

In [12]:
test_question = "熊信寬，藝名熊仔，是臺灣饒舌創作歌⼿。2022年獲得第33屆⾦曲獎最佳作詞⼈獎，2023年獲得第34屆⾦曲獎最佳華語專輯獎。請問熊仔的碩班指導教授為？"
result = await pipeline(test_question)
print("\n[Pipeline 回答結果]:\n", result)

原始問題: 熊信寬，藝名熊仔，是臺灣饒舌創作歌⼿。2022年獲得第33屆⾦曲獎最佳作詞⼈獎，2023年獲得第34屆⾦曲獎最佳華語專輯獎。請問熊仔的碩班指導教授為？
精簡問題: 您好，我幫助你將問題轉換成單一句簡明問：

熊仔的碩班指導教授是誰？
關鍵字: 熊仔的碩士指導教授
Google 搜尋到的 URL】 ['https://web.cheers.com.tw/issue/2016/master/article/article7.html', 'https://zh.wikipedia.org/wiki/%E7%86%8A%E4%BB%94', 'https://www.threads.com/@oahgnehcuil/post/DG43Z2fyeuY/%E7%86%8A%E4%BB%94%E7%9A%84%E6%8C%87%E5%B0%8E%E6%95%99%E6%8E%88%E6%98%AF%E6%9D%8E%E7%90%B3%E5%B1%B1%E6%A8%A1%E5%9E%8B%E6%AF%8F%E6%AC%A1%E5%87%BA%E4%BE%86%E7%9A%84%E7%AD%94%E6%A1%88%E9%83%BD%E4%B8%8D%E4%B8%80%E6%A8%A3%E4%BD%86%E9%83%BD%E6%98%AF%E9%8C%AF%E7%9A%84', 'https://web.cheers.com.tw/issue/2016/master/article/article8.html']
【檢索網址】: https://web.cheers.com.tw/issue/2016/master/article/article7.html [成功]
【檢索網址】: https://zh.wikipedia.org/wiki/%E7%86%8A%E4%BB%94 [成功]
搜尋結果: [('https://web.cheers.com.tw/issue/2016/master/article/article7.html', 'Justamoment...EnableJavaScriptandcookiestocontinue'), ('https://zh.wikipedia.org/wiki/%E7%86%8A%E4%BB%94', '熊仔-维基百科，

In [13]:

test_question = "台北在台灣，在台北這個城市很濕冷，那台北旁邊的基隆市冬天會下雪嗎？"
result = await pipeline(test_question)
print("\n[Pipeline 回答結果]:\n", result)

原始問題: 台北在台灣，在台北這個城市很濕冷，那台北旁邊的基隆市冬天會下雪嗎？
精簡問題: 基隆市冬天會下雪嗎？
關鍵字: 關鍵詞或词组：
基隆市、冬天下雪
Google 搜尋到的 URL】 ['https://www.google.com/search?num=6', '/search?num=6', '']
【檢索網址】: https://www.google.com/search?num=6 [成功]
【檢索網址】: /search?num=6 [失敗]
【檢索網址】:  [失敗]
搜尋結果: [('https://www.google.com/search?num=6', 'GoogleGmailImagesSigninAdvancedsearchAdvertisingBusinessSolutionsAboutGoogle©2025-Privacy-Terms')]
【實際搜尋的 URL】 ['https://www.google.com/search?num=6']
QA 輸入內容: 【問題】基隆市冬天會下雪嗎？
【檢索內容】來源：https://www.google.com/search?num=6
GoogleGmailImagesSigninAdvancedsearchAdvertisingBusinessSolutionsAboutGoogle©2025-Privacy-Terms
最終答案: 根據檢索內容無法判斷。

[Pipeline 回答結果]:
 根據檢索內容無法判斷。


In [14]:
from pathlib import Path

# Fill in your student ID first.
STUDENT_ID = ""

STUDENT_ID = STUDENT_ID.lower()
with open('./public.txt', 'r') as input_f:
    questions = input_f.readlines()
    questions = [l.strip().split(',')[0] for l in questions]
    for id, question in enumerate(questions, 1):
        if Path(f"./{STUDENT_ID}_{id}.txt").exists():
            continue
        answer = await pipeline(question)
        answer = answer.replace('\n',' ')
        print(id, answer)
        with open(f'./{STUDENT_ID}_{id}.txt', 'w') as output_f:
            print(answer, file=output_f)

with open('./private.txt', 'r') as input_f:
    questions = input_f.readlines()
    for id, question in enumerate(questions, 31):
        if Path(f"./{STUDENT_ID}_{id}.txt").exists():
            continue
        answer = await pipeline(question)
        answer = answer.replace('\n',' ')
        print(id, answer)
        with open(f'./{STUDENT_ID}_{id}.txt', 'a') as output_f:
            print(answer, file=output_f)

原始問題: 校歌為學校（包括小學、中學、大學等）宣告或者規定的代表該校的歌曲。用於體現該校的治學理念、辦學理想等學校文化。「虎山雄風飛揚」是哪間學校的校歌歌詞？
精簡問題: 「虎山雄風飛揚」是哪間學校的校歌？
關鍵字: 虎山雄風飛揚、校歌
Google 搜尋到的 URL】 ['https://www.ptt.cc/man/lyrics/DCF1/DD3E/DB30/M.1221557052.A.1FC.html', 'https://www.google.com/search?num=6', 'https://tmec.ntou.edu.tw/var/file/16/1016/img/948/803087035.pdf', 'https://ghps.ntct.edu.tw/p/404-1039-222350.php']
【檢索網址】: https://www.ptt.cc/man/lyrics/DCF1/DD3E/DB30/M.1221557052.A.1FC.html [成功]
【檢索網址】: https://www.google.com/search?num=6 [成功]
搜尋結果: [('https://www.ptt.cc/man/lyrics/DCF1/DD3E/DB30/M.1221557052.A.1FC.html', '校歌南投縣南投市光華國小-精華區lyrics-批踢踢實業坊批踢踢實業坊›精華區betalyrics關於我們聯絡資訊返回上層作者ypx0409(ypx0409)看板lyrics標題校歌南投縣南投市光華國小時間TueSep1603:13:342008貓羅溪旁虎山雄風飛揚這兒是我們生長的地方良師益友濟濟一堂克勤克儉奮發圖強長我育我飲水思源立定志向四海名揚忠心復國是理想光華意志堅光華氣勢昂你我相共勉誓把華夏重光--※發信站:批踢踢實業坊(ptt.cc)◆From:122.116.179.102推vibrant224:國小校歌推...09/1611:10推sognare:校友友情推09/1614:09'), ('https://www.google.com/search?num=6', 'GoogleGmailImagesSigninAdvancedsearchAdvertisingBusinessSolutionsAboutGoogle©2